# pyTorch로 Transformer 간단 구현

In [1]:
!pip install bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 760.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 인코더

##### Self-Attention

In [2]:
# 시각화: 단어와 뉴런 간 연결 분석 시각화 (단일 문장)
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

Output hidden; open in https://colab.research.google.com to view.

In [3]:
# 시각화: 어텐션 헤드 간의 단어 관계 시각화 (문장 간 관계 포함)
from transformers import AutoModel
from bertviz import head_view

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sent1 = "time flies like an arrow"
sent2 = "fruit flies like a banana"

viz_inputs = tokenizer(sent1, sent2, return_tensors="pt")
attention = model(**viz_inputs).attentions
sent2_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sent2_start, heads=[8])

Output hidden; open in https://colab.research.google.com to view.

BERT 모델 (bert-base-uncased) <br>
|---- 12개의 레이어 (Transformer Layers) <br>
|----|---- 12개의 어텐션 헤드 (각 레이어마다) <br>
|----|----|---- 64차원 벡터 출력 (각 헤드마다) <br>
|----|----|---- 12개 헤드의 출력을 합쳐 768차원의 뉴런 벡터(hidden state) 생성

In [4]:
# 토큰화
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [5]:
# Embedding layer 생성
import torch.nn as nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
emb = nn.Embedding(config.vocab_size, config.hidden_size)
emb

Embedding(30522, 768)

In [6]:
# Embedding 처리
inputs_embedded = emb(inputs.input_ids)
inputs_embedded.size()

torch.Size([1, 5, 768])

In [7]:
# Self-Attetion 메커니즘
import torch
import torch.nn.functional as F
from math import sqrt

# 1. score 계산
query = key = value = inputs_embedded
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)

# 2. softmax 적용
weights = F.softmax(scores, dim=-1)

# 3. attetion value 계산
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [8]:
# Self-Attention 함수
def attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)

In [9]:
# Multi-Head Attention

class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_state):
    return attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
    )

In [10]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    heads_num = config.num_attention_heads
    head_dim = embed_dim // heads_num
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(heads_num)]
    )
    self.output_layer = nn.Linear(embed_dim, embed_dim)

  def forward(self, hidden_state):
    output = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
    output = self.output_layer(output)
    return output

In [11]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embedded)
attn_output.size()

torch.Size([1, 5, 768])

##### Feed-Forward

- 간단한 두 개 층으로 구성된 완전 연결 신경망
- 전체 임베딩 시퀀스를 하나의 벡터로 처리하지 않고 각 임베딩을 독립적으로 처리
- GELU(Gaussian Error Linear Unit) 활성 함수 많이 사용

In [12]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)

  def forward(self, x):
    x = self.linear1(x)
    x = self.gelu(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x

In [13]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_output)
ff_outputs.size()

torch.Size([1, 5, 768])

##### 정규화를 반영한 EncoderLayer
- 사전 층 정규화

In [14]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.layer_norm1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm1(x)
    x = x + self.attention(hidden_state)
    x = x + self.feed_forward(self.layer_norm2(x))
    return x

In [15]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embedded.shape, encoder_layer(inputs_embedded).shape

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

##### 위치 임베딩 (Positional Encoding)

In [16]:
class Embeddings(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
    self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
    self.dropout = nn.Dropout()

  def forward(self, inputs_ids):
    seq_length = inputs_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
    token_embeddings = self.token_embeddings(inputs_ids)
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = token_embeddings + position_embeddings
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings

In [17]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

##### 완전한 인코더

In [18]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList([TransformerEncoderLayer(config)
                                  for _ in range(config.num_hidden_layers)])
  def forward(self, x):
    x = self.embeddings(x)
    for layer in self.layers:
      x = layer(x)
    return x

### 디코더

- "Masked" Multi-Head Self-Attention
-  Encoder-Decoder Attention

In [19]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [20]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[28.0123,    -inf,    -inf,    -inf,    -inf],
         [-0.1712, 28.6345,    -inf,    -inf,    -inf],
         [ 0.8116,  0.1485, 27.0649,    -inf,    -inf],
         [ 0.9380,  0.8784,  0.9582, 29.1766,    -inf],
         [-0.2121,  0.1001,  0.8471, -0.5971, 26.7965]]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
# attention + masking
def attention(query, key, value, mask=None):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  if mask is not None:
    scores.masked_fill(mask == 0, -float("inf"))
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)